## Load and Process Dataset 

Dataset source: https://www.kaggle.com/snap/amazon-fine-food-reviews

In [1]:
import csv
from nltk import word_tokenize

import string

summaries = []
texts = []

def clean(text):
    text = text.lower()                           #lower() 로 대문자를 소문자로 변경
    printable = set(string.printable)
    text = "".join(list(filter(lambda x: x in printable, text))) #filter funny characters, if any.  text에 있으면서, printable list에 있는 x를 선택
    return text

text_max_len = 500
text_min_len = 25
summary_max_len = 30
vocab2idx = {}

#Data from https://www.kaggle.com/snap/amazon-fine-food-reviews
with open('Dataset/Reviews.csv') as csvfile:    #text와 summary가 함께 제공되는 csv 파일
                                                #예) summary : Good Quality Dog Food
                                                #   text : I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.
    
    Reviews = csv.DictReader(csvfile)
    
    i=0
    
    for row in Reviews:
        
        text = row['Text']                #review text 하나씩 text에 저장
        summary = row['Summary']          #review summary 하나씩 summary에 저장
        
        if len(text) <= text_max_len and len(text) >= text_min_len and len(summary) <= summary_max_len:   #위에서 지정한 길이에 부합한다면
            #print(i)

            clean_text = clean(text)          #review text를 전처리
            clean_summary = clean(summary)    #review summary를 전처리
            
            tokenized_summary = word_tokenize(clean_summary)   #nltk에서 import한 word_tokenize로 tokenize
            tokenized_text = word_tokenize(clean_text)
            
            # BUILD VOCABULARY
            
            for word in tokenized_text:(
                if word not in vocab2idx:
                    vocab2idx[word]=len(vocab2idx)    #tokenize된 단어들을 순서대로 0번부터 index를 주면서 vocab2idx에 입력, 중복은 없도록
                                                      # 예) {(i:0), (have:1), (bought:2)....}
            for word in tokenized_summary:
                if word not in vocab2idx:
                    vocab2idx[word]=len(vocab2idx)
                    
            ## ________________

            summaries.append(tokenized_summary)       #최종적으로 summaries에는 csv파일에 있는 모든 summary를 설정한 사이즈에 맞고, clean하고, tokenize된 데이터가 들어간다
            texts.append(tokenized_text)

            if i%10000==0:
                print("Processing data # {}".format(i))

            i+=1

print("\n# of Data: {}".format(len(texts)))

Processing data # 0
Processing data # 10000
Processing data # 20000
Processing data # 30000
Processing data # 40000
Processing data # 50000
Processing data # 60000
Processing data # 70000
Processing data # 80000
Processing data # 90000
Processing data # 100000
Processing data # 110000
Processing data # 120000
Processing data # 130000
Processing data # 140000
Processing data # 150000
Processing data # 160000
Processing data # 170000
Processing data # 180000
Processing data # 190000
Processing data # 200000
Processing data # 210000
Processing data # 220000
Processing data # 230000
Processing data # 240000
Processing data # 250000
Processing data # 260000
Processing data # 270000
Processing data # 280000
Processing data # 290000
Processing data # 300000
Processing data # 310000
Processing data # 320000
Processing data # 330000

# of Data: 337465


## Random Sample

In [2]:
import random

index = random.randint(0,len(texts)-1)

print("SAMPLE CLEANED & TOKENIZED TEXT: \n\n{}\n\n".format(texts[index]))
print("SAMPLE CLEANED & TOKENIZED SUMMARY: \n\n{}\n".format(summaries[index]))

SAMPLE CLEANED & TOKENIZED TEXT: 

['we', 'got', 'a', 'box', 'of', 'these', 'to', 'put', 'in', 'gift', 'bags', 'for', 'our', 'sons', "'", 'birthday', 'party', '.', 'what', 'can', 'i', 'say', ':', 'it', "'s", 'a', 'box', 'of', 'gummi', 'pizzas', '.', 'i', 'would', "n't", 'recommend', 'eating', 'a', 'whole', 'box', 'yourself', ',', 'but', 'sharing', 'them', 'with', 'a', 'bunch', 'of', 'kids', 'works', 'great.', '<', 'br', '/', '>', 'from', 'a', 'gummi', 'candy', 'standpoint', ',', 'they', "'re", 'not', 'the', 'yummiest', 'kind', 'of', 'gummi', 'candy', ',', 'but', 'they', 'have', 'a', 'bit', 'more', 'novelty', 'with', 'the', 'pizza', 'appearance', '.']


SAMPLE CLEANED & TOKENIZED SUMMARY: 

['yummi']



## Load Embeddings

Loading pre-trained GloVe embeddings. Source of Data: https://nlp.stanford.edu/projects/glove/

In [3]:
import numpy as np

vocab = []
embd = []
special_tags = ['<UNK>','<PAD>','<EOS>']         #unk:unkown token / pad: 정해진 길이를 채우기 위한 padding / eos:end of sentence


def loadEmbeddings(filename):
    vocab2embd = {}
    
    with open(filename, 'rt', encoding='UTF8') as infile:   #unicode decode error발생하여 encoding='UTF8' 추가     
        for line in infile:
            row = line.strip().split(' ')                   #strip 함수로 양쪽 끝에 있는 공백과 \n 기호를 삭제 / split 함수로 공백 기준으로 line을 나눈 list 생성
            word = row[0].lower()                           #lower 함수로 소문자를 대문자로 바꾸기
            if word not in vocab2embd:
                vocab2embd[word]=np.asarray(row[1:],np.float32)      #ㅠㅠ파일이 너무 커서 안열림. 어떻게 생겼는지 알고싶은데

    print('Embedding Loaded.')
    return vocab2embd

vocab2embd = loadEmbeddings('Embeddings/glove.6B.100d.txt')

for word in vocab2idx:
    if word in vocab2embd:
        vocab.append(word)                              #embedding되는 word만 vocab에 넣음
        embd.append(vocab2embd[word])                   #하나의 word에 대하여 vaocab, embd list의 같은 index위치에 word와 embedding이 있는 것.
        
for special_tag in special_tags:
    vocab.append(special_tag)
    embd.append(np.random.rand(len(embd[0]),))
    
vocab2idx = {word:idx for idx,word in enumerate(vocab)} #이제 word가 key / index가 value  위에서 중복이 없도록 word를 입력했기 때문에 문제없음
embd = np.asarray(embd,np.float32)   

print("Vocabulary Size: {}".format(len(vocab2idx)))
    



Embedding Loaded.
Vocabulary Size: 43544


In [4]:
print(vocab2idx['<EOS>'])

43543


## Vectorize Data

In [5]:
vec_texts=[]
vec_summaries=[]

for text,summary in zip(texts,summaries):
    # Replace out of vocab words with index for '<UNK>' tag
    #get함수는 딕셔너리에서 ()안에 있는 key값에 대응하는 value값을 return
    #a.get(x, y) : a라는 딕셔너리에서 y라는 key값에 대응하는 value값을 return하도록 하는데, 없으면 y를 return
    vec_texts.append([vocab2idx.get(word,vocab2idx['<UNK>']) for word in text])  #word를 int값으로 vectorize
    vec_summaries.append([vocab2idx.get(word,vocab2idx['<UNK>']) for word in summary])

## Shuffle Data

In [6]:
import random
random.seed(101)

texts_idx = [idx for idx in range(len(vec_texts))]
random.shuffle(texts_idx)

vec_texts = [vec_texts[idx] for idx in texts_idx]
vec_summaries = [vec_summaries[idx] for idx in texts_idx]

## Split Data into train, validation, and test sets.

In [7]:
# Use first 10000 data for testing, the next 10000 data for validation, and rest for training

test_summaries = vec_summaries[0:10000]
test_texts = vec_texts[0:10000]

val_summaries = vec_summaries[10000:20000]
val_texts = vec_texts[10000:20000]

train_summaries = vec_summaries[20000:]
train_texts = vec_texts[20000:]

## Bucket And Batch Function

In [8]:
def bucket_and_batch(texts,summaries,batch_size=32):
    
    # Sort summaries and texts according to the length of text
    # (So that texts with similar lengths tend to remain in the same batch and thus require less padding)
    
    text_lens = [len(text) for text in texts]
    sortedidx = np.flip(np.argsort(text_lens),axis=0) #text_lens를 작은 값부터 큰 값 순서대로 sort한 후, 행을 기준으로 뒤집음. 
                                                      #즉, 큰 값이 앞으로, 작은 값이 뒤로 
    texts=[texts[idx] for idx in sortedidx]          #문장이 긴 text가 앞에, 문장이 짧은 text가 뒤에
    summaries=[summaries[idx] for idx in sortedidx]
    
    batches_text=[]
    batches_summary=[]
    batches_true_text_len = []
    batches_true_summary_len = []
    
    i=0
    while i < (len(texts)-batch_size):               #batch size에 맞춰서 padding
        
        max_len = len(texts[i])                       
        
        batch_text=[]
        batch_summary=[]
        batch_true_text_len=[]
        batch_true_summary_len=[]
        
        for j in range(batch_size):
            
            padded_text = texts[i+j]
            padded_summary = summaries[i+j]
            
            batch_true_text_len.append(len(texts[i+j]))
            batch_true_summary_len.append(len(summaries[i+j])+1)
     
            while len(padded_text) < max_len:
                padded_text.append(vocab2idx['<PAD>'])

            padded_summary.append(vocab2idx['<EOS>']) #End of Sentence Marker
            while len(padded_summary) < summary_max_len+1:
                padded_summary.append(vocab2idx['<PAD>'])
            
        
            batch_text.append(padded_text)
            batch_summary.append(padded_summary)
        
        batches_text.append(batch_text)
        batches_summary.append(batch_summary)
        batches_true_text_len.append(batch_true_text_len)
        batches_true_summary_len.append(batch_true_summary_len)
        
        i+=batch_size
        
    return batches_text, batches_summary, batches_true_text_len, batches_true_summary_len

## Prepare Batches

In [9]:
train_batches_text, train_batches_summary, train_batches_true_text_len, train_batches_true_summary_len \
= bucket_and_batch(train_texts, train_summaries)

val_batches_text, val_batches_summary, val_batches_true_text_len, val_batches_true_summary_len \
= bucket_and_batch(val_texts, val_summaries)

test_batches_text, test_batches_summary, test_batches_true_text_len, test_batches_true_summary_len \
= bucket_and_batch(test_texts, test_summaries)

## Save Data

In [10]:
import json

d = {}

d["vocab"] = vocab2idx
d["embd"] = embd.tolist()
d["train_batches_text"] = train_batches_text
d["test_batches_text"] = test_batches_text
d["val_batches_text"] = val_batches_text
d["train_batches_summary"] = train_batches_summary
d["test_batches_summary"] = test_batches_summary
d["val_batches_summary"] = val_batches_summary
d["train_batches_true_text_len"] = train_batches_true_text_len
d["val_batches_true_text_len"] = val_batches_true_text_len
d["test_batches_true_text_len"] = test_batches_true_text_len
d["train_batches_true_summary_len"] = train_batches_true_summary_len
d["val_batches_true_summary_len"] = val_batches_true_summary_len
d["test_batches_true_summary_len"] = test_batches_true_summary_len

with open('Processed_Data/Amazon_Reviews_Processed.json', 'w') as outfile:
    json.dump(d, outfile)
